In [1]:
import dis
from malthusia.engine.container.instrument import Instrument

In [6]:
s = """
def turn():
    try:
        n = get_bytecode()
        log(f"bytecode: {n}")
        turn()
    except (RecursionError, MemoryError):
        raise
    except:
        turn()
turn()
"""
c = compile(s, "source", "exec")

In [7]:
ci = Instrument.instrument(c, replace_builtins=False, instrument=False, instrument_binary_multiply=True, reraise_dangerous_exceptions=True)


In [8]:
c.co_names

('turn',)

In [9]:
new_builtins = {"__builtins__": 
                {"__instrumented_index": lambda x, y: 32, 
                 "type": type, "__safe_type__": type, 
                 "print": print, 
                 "__instrument_binary_multiply__": lambda a,b: print(f"instrumenting {a} * {b}"), 
                 "__build_class__": __build_class__, 
                 "__name__": "DANGEROUS_main",
                 "RecursionError": RecursionError,
                 "MemoryError": MemoryError,
                 "__instrument__": lambda: None
                }
               }

In [ ]:
exec(c, new_builtins)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  [Previous line repeated 996 more times]
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.p

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

...

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "source", line 4, in turn
NameError: name 'get_bytecode' is not defined

During handling of the above exce

    _seen.add(id(exc_value))
RecursionError: maximum recursion depth exceeded while calling a Python object

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 493, in __init__
    context = TracebackException(
  [Previous line repeated 996 more times]
  File "/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/traceback.py", line 476, in __init__
    _seen.add(id(exc_value))
RecursionError: maximum recursion depth exceeded while calling a Python object

During handling of the above e

In [21]:
exec(ci, new_builtins)

In [15]:
dis.dis(c)

  2           0 LOAD_CONST               0 (<code object turn at 0x10c23c920, file "source", line 2>)
              2 LOAD_CONST               1 ('turn')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (turn)
              8 LOAD_CONST               2 (None)
             10 RETURN_VALUE

Disassembly of <code object turn at 0x10c23c920, file "source", line 2>:
  3           0 SETUP_FINALLY           30 (to 32)

  4           2 LOAD_GLOBAL              0 (get_bytecode)
              4 CALL_FUNCTION            0
              6 STORE_FAST               0 (n)

  5           8 LOAD_GLOBAL              1 (log)
             10 LOAD_CONST               1 ('bytecode: ')
             12 LOAD_FAST                0 (n)
             14 FORMAT_VALUE             0
             16 BUILD_STRING             2
             18 CALL_FUNCTION            1
             20 POP_TOP

  6          22 LOAD_GLOBAL              2 (turn)
             24 CALL_FUNCTION            

In [9]:
dis.dis(ci)

  2           0 LOAD_CONST               0 (<code object turn at 0x10c1a5920, file "source", line 2>)
              2 LOAD_CONST               1 ('turn')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (turn)
              8 LOAD_CONST               2 (None)
             10 RETURN_VALUE

Disassembly of <code object turn at 0x10c1a5920, file "source", line 2>:
  3           0 SETUP_FINALLY           30 (to 32)

  4           2 LOAD_GLOBAL              0 (get_bytecode)
              4 CALL_FUNCTION            0
              6 STORE_FAST               0 (n)

  5           8 LOAD_GLOBAL              1 (log)
             10 LOAD_CONST               1 ('bytecode: ')
             12 LOAD_FAST                0 (n)
             14 FORMAT_VALUE             0
             16 BUILD_STRING             2
             18 CALL_FUNCTION            1
             20 POP_TOP

  6          22 LOAD_GLOBAL              2 (turn)
             24 CALL_FUNCTION            

In [13]:
s1 = """  2           0 LOAD_GLOBAL              8 (__instrument__)
              2 CALL_FUNCTION            0
              4 POP_TOP
              6 LOAD_CONST               0 ('hej')
              8 LOAD_GLOBAL              8 (__instrument__)
             10 CALL_FUNCTION            0
             12 POP_TOP
             14 STORE_NAME               0 (x)

  3          16 LOAD_GLOBAL              8 (__instrument__)
             18 CALL_FUNCTION            0
             20 POP_TOP
             22 SETUP_FINALLY           48 (to 72)

  4          24 LOAD_GLOBAL              8 (__instrument__)
             26 CALL_FUNCTION            0
             28 POP_TOP
             30 LOAD_CONST               1 (12)
             32 LOAD_GLOBAL              8 (__instrument__)
             34 CALL_FUNCTION            0
             36 POP_TOP
             38 LOAD_CONST               2 (0)
             40 LOAD_GLOBAL              8 (__instrument__)
             42 CALL_FUNCTION            0
             44 POP_TOP
             46 BINARY_TRUE_DIVIDE
             48 LOAD_GLOBAL              8 (__instrument__)
             50 CALL_FUNCTION            0
             52 POP_TOP
             54 STORE_NAME               0 (x)
             56 LOAD_GLOBAL              8 (__instrument__)
             58 CALL_FUNCTION            0
             60 POP_TOP
             62 POP_BLOCK
             64 LOAD_GLOBAL              8 (__instrument__)
             66 CALL_FUNCTION            0
             68 POP_TOP
             70 JUMP_FORWARD           104 (to 176)
        >>   72 DUP_TOP
             74 LOAD_NAME                5 (RecursionError)
             76 LOAD_NAME                6 (MemoryError)
             78 BUILD_TUPLE              2
             80 COMPARE_OP              10 (exception match)
             82 POP_JUMP_IF_FALSE       96
             84 POP_TOP
             86 POP_TOP
             88 POP_TOP
             90 RAISE_VARARGS            0
             92 POP_EXCEPT
             94 JUMP_FORWARD            80 (to 176)

  5     >>   96 LOAD_GLOBAL              8 (__instrument__)
             98 CALL_FUNCTION            0
            100 POP_TOP
            102 POP_TOP
            104 LOAD_GLOBAL              8 (__instrument__)
            106 CALL_FUNCTION            0
            108 POP_TOP
            110 POP_TOP
            112 LOAD_GLOBAL              8 (__instrument__)
            114 CALL_FUNCTION            0
            116 POP_TOP
            118 POP_TOP

  6         120 LOAD_GLOBAL              8 (__instrument__)
            122 CALL_FUNCTION            0
            124 POP_TOP
            126 LOAD_NAME                1 (print)
            128 LOAD_GLOBAL              8 (__instrument__)
            130 CALL_FUNCTION            0
            132 POP_TOP
            134 LOAD_CONST               3 ('ohno')
            136 LOAD_GLOBAL              8 (__instrument__)
            138 CALL_FUNCTION            0
            140 POP_TOP
            142 CALL_FUNCTION            1
            144 LOAD_GLOBAL              8 (__instrument__)
            146 CALL_FUNCTION            0
            148 POP_TOP
            150 POP_TOP
            152 LOAD_GLOBAL              8 (__instrument__)
            154 CALL_FUNCTION            0
            156 POP_TOP
            158 POP_EXCEPT
            160 LOAD_GLOBAL              8 (__instrument__)
            162 CALL_FUNCTION            0
            164 POP_TOP
            166 JUMP_FORWARD             8 (to 176)
            168 LOAD_GLOBAL              8 (__instrument__)
            170 CALL_FUNCTION            0
            172 POP_TOP
            174 END_FINALLY

  7     >>  176 LOAD_GLOBAL              8 (__instrument__)
            178 CALL_FUNCTION            0
            180 POP_TOP
            182 LOAD_CONST               4 (3)
            184 LOAD_GLOBAL              8 (__instrument__)
            186 CALL_FUNCTION            0
            188 POP_TOP
            190 STORE_NAME               0 (x)

  8         192 LOAD_GLOBAL              8 (__instrument__)
            194 CALL_FUNCTION            0
            196 POP_TOP
            198 LOAD_CONST               5 (4)
            200 LOAD_GLOBAL              8 (__instrument__)
            202 CALL_FUNCTION            0
            204 POP_TOP
            206 STORE_NAME               2 (y)

  9         208 LOAD_GLOBAL              8 (__instrument__)
            210 CALL_FUNCTION            0
            212 POP_TOP
            214 LOAD_NAME                0 (x)
            216 LOAD_GLOBAL              8 (__instrument__)
            218 CALL_FUNCTION            0
            220 POP_TOP
            222 LOAD_NAME                2 (y)
            224 LOAD_GLOBAL              8 (__instrument__)
            226 CALL_FUNCTION            0
            228 POP_TOP
            230 BINARY_ADD
            232 LOAD_GLOBAL              8 (__instrument__)
            234 CALL_FUNCTION            0
            236 POP_TOP
            238 STORE_NAME               3 (z)
            240 LOAD_GLOBAL              8 (__instrument__)
            242 CALL_FUNCTION            0
            244 POP_TOP
            246 LOAD_CONST               6 (None)
            248 LOAD_GLOBAL              8 (__instrument__)
            250 CALL_FUNCTION            0
            252 POP_TOP
            254 RETURN_VALUE
"""

In [14]:
s2="""  2           0 LOAD_GLOBAL              7 (__instrument__)
              2 CALL_FUNCTION            0
              4 POP_TOP
              6 LOAD_CONST               0 ('hej')
              8 LOAD_GLOBAL              7 (__instrument__)
             10 CALL_FUNCTION            0
             12 POP_TOP
             14 STORE_NAME               0 (x)

  3          16 LOAD_GLOBAL              7 (__instrument__)
             18 CALL_FUNCTION            0
             20 POP_TOP
             22 SETUP_FINALLY           48 (to 72)

  4          24 LOAD_GLOBAL              7 (__instrument__)
             26 CALL_FUNCTION            0
             28 POP_TOP
             30 LOAD_CONST               1 (12)
             32 LOAD_GLOBAL              7 (__instrument__)
             34 CALL_FUNCTION            0
             36 POP_TOP
             38 LOAD_CONST               2 (0)
             40 LOAD_GLOBAL              7 (__instrument__)
             42 CALL_FUNCTION            0
             44 POP_TOP
             46 BINARY_TRUE_DIVIDE
             48 LOAD_GLOBAL              7 (__instrument__)
             50 CALL_FUNCTION            0
             52 POP_TOP
             54 STORE_NAME               0 (x)
             56 LOAD_GLOBAL              7 (__instrument__)
             58 CALL_FUNCTION            0
             60 POP_TOP
             62 POP_BLOCK
             64 LOAD_GLOBAL              7 (__instrument__)
             66 CALL_FUNCTION            0
             68 POP_TOP
             70 JUMP_FORWARD           176 (to 248)

  5     >>   72 LOAD_GLOBAL              7 (__instrument__)
             74 CALL_FUNCTION            0
             76 POP_TOP
             78 DUP_TOP
             80 LOAD_GLOBAL              7 (__instrument__)
             82 CALL_FUNCTION            0
             84 POP_TOP
             86 LOAD_NAME                1 (RecursionError)
             88 LOAD_GLOBAL              7 (__instrument__)
             90 CALL_FUNCTION            0
             92 POP_TOP
             94 LOAD_NAME                2 (MemoryError)
             96 LOAD_GLOBAL              7 (__instrument__)
             98 CALL_FUNCTION            0
            100 POP_TOP
            102 BUILD_TUPLE              2
            104 LOAD_GLOBAL              7 (__instrument__)
            106 CALL_FUNCTION            0
            108 POP_TOP
            110 COMPARE_OP              10 (exception match)
            112 LOAD_GLOBAL              7 (__instrument__)
            114 CALL_FUNCTION            0
            116 POP_TOP
            118 POP_JUMP_IF_FALSE      168
            120 LOAD_GLOBAL              7 (__instrument__)
            122 CALL_FUNCTION            0
            124 POP_TOP
            126 POP_TOP
            128 LOAD_GLOBAL              7 (__instrument__)
            130 CALL_FUNCTION            0
            132 POP_TOP
            134 POP_TOP
            136 LOAD_GLOBAL              7 (__instrument__)
            138 CALL_FUNCTION            0
            140 POP_TOP
            142 POP_TOP

  6         144 LOAD_GLOBAL              7 (__instrument__)
            146 CALL_FUNCTION            0
            148 POP_TOP
            150 RAISE_VARARGS            0
            152 LOAD_GLOBAL              7 (__instrument__)
            154 CALL_FUNCTION            0
            156 POP_TOP
            158 POP_EXCEPT
            160 LOAD_GLOBAL              7 (__instrument__)
            162 CALL_FUNCTION            0
            164 POP_TOP
            166 JUMP_FORWARD            80 (to 248)

  7     >>  168 LOAD_GLOBAL              7 (__instrument__)
            170 CALL_FUNCTION            0
            172 POP_TOP
            174 POP_TOP
            176 LOAD_GLOBAL              7 (__instrument__)
            178 CALL_FUNCTION            0
            180 POP_TOP
            182 POP_TOP
            184 LOAD_GLOBAL              7 (__instrument__)
            186 CALL_FUNCTION            0
            188 POP_TOP
            190 POP_TOP

  8         192 LOAD_GLOBAL              7 (__instrument__)
            194 CALL_FUNCTION            0
            196 POP_TOP
            198 LOAD_NAME                3 (print)
            200 LOAD_GLOBAL              7 (__instrument__)
            202 CALL_FUNCTION            0
            204 POP_TOP
            206 LOAD_CONST               3 ('ohno')
            208 LOAD_GLOBAL              7 (__instrument__)
            210 CALL_FUNCTION            0
            212 POP_TOP
            214 CALL_FUNCTION            1
            216 LOAD_GLOBAL              7 (__instrument__)
            218 CALL_FUNCTION            0
            220 POP_TOP
            222 POP_TOP
            224 LOAD_GLOBAL              7 (__instrument__)
            226 CALL_FUNCTION            0
            228 POP_TOP
            230 POP_EXCEPT
            232 LOAD_GLOBAL              7 (__instrument__)
            234 CALL_FUNCTION            0
            236 POP_TOP
            238 JUMP_FORWARD             8 (to 248)
            240 LOAD_GLOBAL              7 (__instrument__)
            242 CALL_FUNCTION            0
            244 POP_TOP
            246 END_FINALLY

  9     >>  248 LOAD_GLOBAL              7 (__instrument__)
            250 CALL_FUNCTION            0
            252 POP_TOP
            254 LOAD_CONST               4 (3)
            256 LOAD_GLOBAL              7 (__instrument__)
            258 CALL_FUNCTION            0
            260 POP_TOP
            262 STORE_NAME               0 (x)

 10         264 LOAD_GLOBAL              7 (__instrument__)
            266 CALL_FUNCTION            0
            268 POP_TOP
            270 LOAD_CONST               5 (4)
            272 LOAD_GLOBAL              7 (__instrument__)
            274 CALL_FUNCTION            0
            276 POP_TOP
            278 STORE_NAME               4 (y)

 11         280 LOAD_GLOBAL              7 (__instrument__)
            282 CALL_FUNCTION            0
            284 POP_TOP
            286 LOAD_NAME                0 (x)
            288 LOAD_GLOBAL              7 (__instrument__)
            290 CALL_FUNCTION            0
            292 POP_TOP
            294 LOAD_NAME                4 (y)
            296 LOAD_GLOBAL              7 (__instrument__)
            298 CALL_FUNCTION            0
            300 POP_TOP
            302 BINARY_ADD
            304 LOAD_GLOBAL              7 (__instrument__)
            306 CALL_FUNCTION            0
            308 POP_TOP
            310 STORE_NAME               5 (z)
            312 LOAD_GLOBAL              7 (__instrument__)
            314 CALL_FUNCTION            0
            316 POP_TOP
            318 LOAD_CONST               6 (None)
            320 LOAD_GLOBAL              7 (__instrument__)
            322 CALL_FUNCTION            0
            324 POP_TOP
            326 RETURN_VALUE
"""